<a href="https://colab.research.google.com/github/tony1966/colab/blob/main/reinforcement_learning_ch3_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline

In [3]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Activation, Add, BatchNormalization 
from tensorflow.keras.layers import Conv2D, Dense
from tensorflow.keras.layers import GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [4]:
(train_images, train_labels), (test_images, test_labels)=cifar10.load_data()

170508288/170498071 [==============================] - 3s 0us/step


In [5]:
train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


In [6]:
# 建構卷積層
def conv(filters, kernel_size, strides=1):
    return Conv2D(filters, kernel_size, strides=strides, padding='same', 
            use_bias=True, kernel_initializer='he_normal', 
            kernel_regularizer=l2(0.0001))

In [7]:
# 建構殘差塊 A
def first_residual_unit(filters, strides):
    def f(x):
        # 正規化 → ReLU 激活
        x=BatchNormalization()(x)
        x_b=Activation('relu')(x)
        # 卷積層 → 正規化 → ReLU 激活
        x=conv(filters // 4, 1, strides)(x_b)
        x=BatchNormalization()(x)
        x=Activation('relu')(x)        
        # 卷積層 → 正規化 → ReLU 激活
        x=conv(filters // 4, 3)(x)
        x=BatchNormalization()(x)
        x=Activation('relu')(x)
        # 卷積層
        x=conv(filters, 1)(x)
        # 調整捷徑之 shape 的尺寸
        x_b=conv(filters, 1, strides)(x_b)
        # Add
        return Add()([x, x_b])
    return f

In [8]:
# 建構殘差塊 B
def residual_unit(filters):
    def f(x):
        x_b = x        
        # → 正規化 → ReLU 激活
        x=BatchNormalization()(x)
        x=Activation('relu')(x)        
        # 卷積層 → 正規化 → ReLU 激活
        x=conv(filters // 4, 1)(x)
        x=BatchNormalization()(x)
        x=Activation('relu')(x)        
        # 卷積層 → 正規化 → ReLU 激活
        x=conv(filters // 4, 3)(x)
        x=BatchNormalization()(x)
        x=Activation('relu')(x)        
        # 卷積層
        x=conv(filters, 1)(x)
        # Add
        return Add()([x, x_b])
    return f

In [9]:
# 建構殘差塊 A×1 與 殘差塊 B×17
def residual_block(filters, strides, unit_size):
    def f(x):
        x=first_residual_unit(filters, strides)(x) # 1 個殘差塊 A
        for i in range(unit_size-1):       # 17 個殘差塊 B
            x=residual_unit(filters)(x)
        return x
    return f

In [10]:
# 輸入層
input=Input(shape=(32,32, 3)) # CIFAR 圖片 32x32, RGB 三通道
# 卷積層
x=conv(16, 3)(input) # 16 個 3x3 卷積核
# 殘差塊 x 54
x=residual_block(64, 1, 18)(x)   # 第一組殘差塊
x=residual_block(128, 2, 18)(x)  # 第二組殘差塊
x=residual_block(256, 2, 18)(x)  # 第三組殘差塊
# → 正規化 → ReLU 激活
x=BatchNormalization()(x)
x=Activation('relu')(x)
# 池化層
x=GlobalAveragePooling2D()(x)
# 密集層
output=Dense(10, activation='softmax', kernel_regularizer=l2(0.0001))(x)
# 建構模型
model=Model(inputs=input, outputs=output)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

In [12]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(momentum=0.9), 
      metrics=['acc'])

In [13]:
# 設定訓練集的 ImageDataGenerator
train_gen=ImageDataGenerator( 
    featurewise_center=True, 
    featurewise_std_normalization=True,
    width_shift_range=0.125, 
    height_shift_range=0.125, 
    horizontal_flip=True)
# 設定測試集的 ImageDataGenerator
test_gen=ImageDataGenerator(
    featurewise_center=True, 
    featurewise_std_normalization=True)
for data in (train_gen, test_gen):
    data.fit(train_images)

In [14]:
# 設定 LearningRateScheduler
def step_decay(epoch): # 以 epoch 為單位傳回新的學習率
    x=0.1  # 初始學習率
    if epoch >= 80: x=0.01  # 第 80 週期後學習率降為 0.01
    if epoch >= 120: x=0.001 # 第 120 週期後學習率降為 0.001
    return x
lr_decay=LearningRateScheduler(step_decay, verbose = 1) # 指定學習率指配函式

In [ ]:
# 訓練模型
batch_size=128
history=model.fit(
    train_gen.flow(train_images, train_labels, batch_size=batch_size),
    epochs=120,
    steps_per_epoch=train_images.shape[0] // batch_size,
    validation_data=test_gen.flow(test_images, test_labels, 
                   batch_size=batch_size),
    validation_steps=test_images.shape[0] // batch_size,
    callbacks=[lr_decay])


Epoch 1: LearningRateScheduler setting learning rate to 0.1.
Epoch 1/120
390/390 [==============================] - 150s 316ms/step - loss: 4.5365 - acc: 0.3666 - val_loss: 4.2972 - val_acc: 0.4253 - lr: 0.1000

Epoch 2: LearningRateScheduler setting learning rate to 0.1.
Epoch 2/120
390/390 [==============================] - 127s 325ms/step - loss: 3.7190 - acc: 0.5487 - val_loss: 3.7029 - val_acc: 0.5171 - lr: 0.1000

Epoch 3: LearningRateScheduler setting learning rate to 0.1.
Epoch 3/120
390/390 [==============================] - 125s 319ms/step - loss: 3.1798 - acc: 0.6336 - val_loss: 3.4535 - val_acc: 0.5363 - lr: 0.1000

Epoch 4: LearningRateScheduler setting learning rate to 0.1.
Epoch 4/120
390/390 [==============================] - 125s 321ms/step - loss: 2.7532 - acc: 0.6946 - val_loss: 3.0118 - val_acc: 0.5874 - lr: 0.1000
Epoch 5/120
390/390 [==============================] - 125s 321ms/step - loss: 2.4200 - acc: 0.7293 - val_loss: 2.4970 - val_acc: 0.6753 - lr: 0.1000

E

In [ ]:
# 儲存模型
model.save('resnet.h5')

In [ ]:
# 繪製圖形
plt.plot(history.history['acc'], label='acc')
plt.plot(history.history['val_acc'], label='val_acc')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='best')
plt.show()

In [ ]:
# 評估
test_loss, test_acc=model.evaluate(test_gen.flow(test_images, test_labels))
print('loss: {:.3f}\nacc: {:.3f}'.format(test_loss, test_acc ))

In [ ]:
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(test_images[i])
plt.show()

test_predictions=model.predict(test_gen.flow(test_images[0:10], shuffle=False))
test_predictions=np.argmax(test_predictions, axis=1)
labels=['airplane', 'automobile', 'bird', 'cat', 'deer', 
    'dog', 'frog', 'horse', 'ship', 'truck']
print('前 10 筆預測標籤:',[labels[n] for n in test_predictions])
test_ans=np.argmax(test_labels[:10], axis=1)
print('前 10 筆原始標籤:',[labels[n] for n in test_ans])